In [1]:
import sys
import requests
from time import sleep

from tqdm import tqdm
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import gensim

In [2]:
sys.path.append('../')

In [3]:
from src import settings

In [4]:
client = MongoClient(settings.MONGODB_SETTINGS['host'])
db = client[settings.MONGODB_SETTINGS['db']]
col = db[settings.MONGODB_SETTINGS['collection']]

In [7]:
from src.models import make_texts_corpus
from src.utils import editorJs_data_to_text
import json
from gensim.matutils import cossim
from gensim import similarities

In [9]:
id2word = gensim.corpora.Dictionary.load(settings.PATH_DICTIONARY)
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
lda_model = gensim.models.ldamodel.LdaModel.load('../models/LDA.model')

INFO : loading Dictionary object from ../models/id2word.dictionary
INFO : Dictionary lifecycle event {'fname': '../models/id2word.dictionary', 'datetime': '2022-12-29T00:58:52.474833', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 149 documents, 3659 features, 29716 non-zero entries
INFO : loading LdaModel object from ../models/LDA.model
INFO : loading expElogbeta from ../models/LDA.model.expElogbeta.npy with mmap=None
INFO : setting ignored attribute id2word to None
INFO : setting ignored attribute state to None
INFO : setting ignored attribute dispatcher to None
INFO : LdaModel lifecycle event {'fname': '../models/LDA.model', 'datetime': '2022-12-29T00:58:52.514819', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.

In [71]:
index = similarities.MatrixSimilarity(lda_model[corpus])

INFO : creating matrix with 149 documents and 52 features


In [31]:
for post in col.find():
    print(post['title'], post['idrs'] if 'idrs' in post else False)

ĐỒ ÁN THIẾT KẾ WEBSITE BLOG SITE 0
setState trong React 1
Regular Expression trong Javascript 2
Các bài tập java cơ bản 3
Những câu hỏi lập trình khi pv (2): bài toán ATM 4
Ví dụ sắp xếp nổi bọt với Java 5
Chương trình lọc thư rác bằng Java, code ví dụ 6
Đọc ghi file Excel trong Java sử dụng Apache POI 7
THREAD AND THREAD POOLS IN JAVA 8
Sử dụng CountDownLatch trong Java 9
Multithreading: Java Synchronized Blocks 10
Understand synchronized and improve lock performance in Java 11
Front-End Web Developers 12
Material Design for Web Design 13
Những kỹ năng cần phải có của một Web Designer 14
Rest API - cách Frontend và Backend nói chuyện với nhau qua ví dụ cụ thể 15
Điều cần chú ý để xây dựng RESTful API 16
Dựng một API RESTful đơn giản sử dụng Express và MongoDB 17
xây dựng API với Napa 18
Trí tuệ nhân tạo với Swift 19
Lợi ích và nguy cơ của trí tuệ nhân tạo (AI) 20
Liệu chúng ta có thể xây dựng trí tuệ nhân tạo mà không mất kiểm soát đối với nó? 21
Agile Marketing khái niệm và tại sao n

In [65]:
# main_post = col.find_one({"_id": ObjectId("6388c04363cdee37de74f09c")})
main_post = col.find_one({"idrs": 63})
main_post['title']

'Bắt nhịp lối sống lành mạnh&nbsp;với 4 bước siêu đơn giản'

In [66]:
content = editorJs_data_to_text(json.loads(main_post["content"]))
text_corpus = make_texts_corpus([content])
bow = id2word.doc2bow(next(text_corpus))

In [72]:
vector_doc = lda_model[bow]

In [73]:
sims = index[vector_doc]

In [74]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
#for doc_position, doc_score in sims:
#    print(doc_score, doc_position)

1.0 62
0.9569841 64
0.95598525 61
0.9410794 67
0.9388191 63
0.92625475 60
0.83459336 70
0.8276344 40
0.82686 65
0.7961886 66
0.7127844 126
0.6382414 122
0.60886633 69
0.55982834 117
0.54041946 68
0.52808917 124
0.50037295 44
0.45623726 87
0.44219244 91
0.43932498 84
0.41075248 110
0.37752873 89
0.37284842 88
0.36320695 121
0.35971323 114
0.35961446 129
0.3532535 93
0.35009074 113
0.32863122 90
0.3207724 128
0.31799033 127
0.31295455 95
0.30943787 79
0.3078081 123
0.29296464 98
0.27886888 78
0.27167967 85
0.2714176 92
0.2636469 100
0.2586028 38
0.25546193 20
0.2527218 125
0.24128684 96
0.23800273 105
0.23738849 74
0.2351241 99
0.23327583 132
0.23029971 135
0.23016998 109
0.22905378 131
0.22608566 133
0.21606988 15
0.21589608 106
0.21507066 19
0.21479362 130
0.21345164 81
0.21041764 134
0.20161328 138
0.20094344 144
0.19678092 80
0.19439903 94
0.19275211 83
0.186013 35
0.18591002 75
0.18509579 82
0.1845845 104
0.18359569 147
0.18237329 107
0.1821957 143
0.17258704 139
0.16361147 148
0.16

In [76]:
print(main_post['title'])
i=0
for idrs, distance in sims:
    article = col.find_one({"idrs": idrs})
    print(i, idrs, distance, article['title'])
    i+=1
    if(i==10): break
    

Bắt nhịp lối sống lành mạnh&nbsp;với 4 bước siêu đơn giản
0 62 1.0 Những lợi ích bất ngờ khi bạn duy trì lối sống lành mạnh
1 64 0.9569841 Lối sống lành mạnh có thể làm giảm nguy cơ sa sút trí tuệ và kéo dài tuổi thọ
2 61 0.95598525 Duy trì lối sống lành mạnh nâng cao sức khỏe
3 67 0.9410794 8 cách ăn uống lành mạnh và giúp bạn có cuộc sống khỏe mạnh hơn
4 63 0.9388191 Bắt nhịp lối sống lành mạnh&nbsp;với 4 bước siêu đơn giản
5 60 0.92625475 Khủng hoảng tiền số: Tái ông thất mã?
6 70 0.83459336 11 chế độ ăn uống lành mạnh nhất năm 2020
7 40 0.8276344 Top 15 ngôn ngữ lập trình phổ biến nhất thế giới, bạn đã biết hết chưa?
8 65 0.82686 10 cách duy trì lối sống lành mạnh ai cũng có thể thực hiện
9 66 0.7961886 Chế độ ăn uống lành mạnh là gì và làm cách nào để bắt đầu?


In [78]:
arr = [tup[0] for tup in sims[:10]]
if 'idrs' in main_post: arr.remove(main_post['idrs'])
arr

[62, 64, 61, 67, 60, 70, 40, 65, 66]